In [67]:
import Data.Monoid
import Control.Monad
import Test.QuickCheck

<h3>New Maybe type</h3>

In [53]:
data Optional a = Nada | Only a deriving (Eq, Show)

In [54]:
instance Semigroup a => Semigroup (Optional a) where
    (<>) Nada a = a
    (<>) a Nada = a
    (<>) (Only a) (Only b) = Only (a <> b)

In [55]:
instance Monoid a => Monoid (Optional a) where
    mempty = Only mempty
    mappend = (<>)

<h3>Quick Check examples</h3>

In [30]:
monoidAssoc :: (Eq m, Monoid m)
    => m -> m -> m -> Bool
monoidAssoc a b c = (a <> (b <> c)) == ((a <> b) <> c)

In [41]:
type S = String
quickCheck (monoidAssoc :: S -> S -> S -> Bool)

+++ OK, passed 100 tests.

In [42]:
monoidLeftIdentity :: (Eq m, Monoid m)
    => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m)
    => m -> Bool
monoidRightIdentity a = (a <> mempty) == a

In [43]:
quickCheck (monoidLeftIdentity :: S -> Bool)

+++ OK, passed 100 tests.

In [44]:
quickCheck (monoidRightIdentity :: S -> Bool)

+++ OK, passed 100 tests.

In [46]:
data Bull = Fools | Twoo deriving (Eq, Show)

instance Arbitrary Bull where
    arbitrary = frequency [(1, return Fools), (1, return Twoo)]

instance Semigroup Bull where
    (<>) _ _ = Fools

instance Monoid Bull where
    mempty = Fools
    mappend _ _ = Fools

In [47]:
type BullMappend = Bull -> Bull -> Bull -> Bool

In [48]:
quickCheck (monoidAssoc :: BullMappend)

+++ OK, passed 100 tests.

In [50]:
quickCheck (monoidLeftIdentity :: Bull -> Bool)

*** Failed! Falsifiable (after 4 tests):
Twoo

<h3>Maybe monoid without underlying monoid type</h3>

In [58]:
newtype First' a = First' { getFirst' :: Optional a } deriving (Eq, Show)

In [86]:
instance Arbitrary a => Arbitrary (First' a) where
    arbitrary = frequency [(1, return (First' Nada)), (3, fmap (First' . Only) arbitrary)]

In [93]:
instance Semigroup (First' a) where
    (<>) _ (First' (Only a)) = First' (Only a)
    (<>) a (First' Nada) = a
    
instance Monoid (First' a) where
    mempty = First' Nada
    mappend = (<>)

In [94]:
firstMappend :: First' a -> First' a -> First' a
firstMappend = mappend

In [95]:
type FirstMappend = First' String -> First' String -> First' String -> Bool
type FstId = First' String -> Bool

In [96]:
quickCheck(monoidAssoc :: FirstMappend)

+++ OK, passed 100 tests.

In [97]:
quickCheck(monoidLeftIdentity :: FstId)
quickCheck(monoidRightIdentity :: FstId)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

<h3>List type</h3>

In [100]:
data Listy a = Nil | Cons {head':: a, tail':: Listy a} deriving (Eq, Show)

In [101]:
instance Arbitrary a => Arbitrary (Listy a) where
    arbitrary = frequency [(1, pure Nil), (5, Cons <$> arbitrary <*> arbitrary)]

In [102]:
instance Semigroup (Listy a) where
    (<>) Nil ys = ys
    (<>) (Cons x xs) ys = Cons x (xs <> ys)

In [103]:
instance Monoid (Listy a) where
    mempty = Nil
    mappend = (<>)

In [105]:
type ListyMappend = Listy String -> Listy String -> Listy String -> Bool
type ListyId = Listy String -> Bool

In [106]:
quickCheck(monoidAssoc :: ListyMappend)

+++ OK, passed 100 tests.

In [108]:
quickCheck(monoidLeftIdentity :: ListyId)
quickCheck(monoidRightIdentity :: ListyId)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.